<a href="https://colab.research.google.com/github/HisameOgasahara/HF_transformers/blob/main/HFHT_4_1_Tokenizers_%EB%9D%BC%EC%9D%B4%EB%B8%8C%EB%9F%AC%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### 참고로 이전 노트북에서는 from transformers import BertTokenizer/AutoTokenizer를 했지 Tokenizers 라이브러리를 쓰진 않았다.

#### 간단히 쓸거면 from transformers import AutoTokenizer를 쓰는 게 편하나, 세밀한 제어나 빠른 토큰화가 필요한 경우 Tokenizer를 써서 토크나이저를 훈련하고 transformers에 통합하기도 한다.

In [ ]:
!pip install datasets==2.20.0 transformers==4.41.2

# 4.1.1 Tokenizers 학습

### 데이터셋 준비

In [ ]:
from datasets import load_dataset

dataset = load_dataset("klue", "ynat")
dataset["train"][0]

In [ ]:
target_key = "title"
for key in dataset.column_names.keys():
  with open(f"/content/tokenizer_data_{key}.txt", "w") as f:
    f.write("\n".join(dataset[key][target_key]))

### 사용자 지정 토큰/특수 토큰 및 OOV 토큰 처리

In [ ]:
user_defined_symbols = [
    "[PAD]",  # 문장의 길이를 맞추기 위해 사용되는 토큰
    "[UNK]",  # 토크나이저가 인식할 수 없는 토큰
    "[CLS]",  # bert 계열 모델에서 문장 전체의 정보를 저장하는 토큰
    "[SEP]",  # bert 계열 모델에서 문장 구분을 위해 사용하는 토큰
    "[MASK]", # MLM 모델에서 토큰 마스킹을 위해 사용하는 토큰
]

unused_token_num = 100
unused_list = [f"[UNUSED{i}]" for i in range(unused_token_num)]  # 사전학습 시, 어휘에 없는 토큰을 추가하기 위한 빈 공간

whole_user_defined_symbols = user_defined_symbols + unused_list
print(whole_user_defined_symbols[:10])

### subword 토크나이징 방식 중 하나인 WordPiece를 이용하는 토크나이저 생성, 노멀라이저 생성, pre_토크나이저 생성


#### 참고로 word/character/morpheme 토큰화가 그 이전에 존재했으나 이후 BPE/wordpiece 등 서브워드 방식이 주류가 되었다. 내용이 방대하니 레퍼를 찾아보는거로

In [ ]:
from tokenizers import Tokenizer
from tokenizers.models import WordPiece

bert_tokenizer = Tokenizer(WordPiece(unk_token="[UNK]")) #tokenizers 라이브러리에서 Tokenizer 객체를 생성합니다. WordPiece 모델을 사용하고, 알 수 없는 토큰(unk_token)을 "[UNK]"로 설정

In [ ]:
from tokenizers import normalizers #노말라이저는 구두점 처리, 대소변환, 공백제거 등의 역할 수행(노멀라이저 종류에 따라 다양)

normalizer = normalizers.BertNormalizer() #BertNormalizer 객체 생성: BERT 모델에 특화된 정규화 수행
bert_tokenizer.normalizer = normalizer #위의 cell에서 생성한 bert_tokenizer 객체의 normalizer 속성에 윗줄에서 생성한 normalizer 객체를 할당합니다. 이렇게 하면 bert_tokenizer가 텍스트를 토큰화하기 전에 normalizer에 정의된 정규화 규칙을 적용하게 됩니다.

normalizer.normalize_str("Héllò hôwWnare ü? ") #Héllò -> hello: 소문자 변환, 악센트 제거

In [ ]:
from tokenizers.pre_tokenizers import Whitespace #Whitespace pre-tokenizer를 사용하여 공백을 기준으로 텍스트를 분리합니다. 이는 WordPiece 토큰화 전에 단어 수준의 분리를 수행하는 단계

pre_tokenizer = Whitespace()
bert_tokenizer.pre_tokenizer = pre_tokenizer

pre_tokenizer.pre_tokenize_str("안녕하세요. 제대로 인코딩이 되는지 확인 중입니다.")

#### 왜 wordpiece를 하기전에 whitespace를 이용하는가?

##### - 긴 문자열 처리의 비효율성: WordPiece는 기본적으로 문자 단위에서 시작하여 점진적으로 토큰을 병합합니다. 만약 매우 긴 단어나 공백 없이 이어진 문자열이 입력되면, WordPiece 알고리즘은 모든 가능한 문자 조합을 고려해야 하므로 계산량이 매우 커지고, 훈련 속도가 느려집니다.

##### - 잘못된 토큰화 가능성: 띄어쓰기가 없는 문장이나, 오타가 있는 경우, WordPiece가 의도하지 않은 방식으로 토큰을 병합할 수 있습니다. 예를 들어, "thisisaword"가 "this", "isa", "word"로 분리되지 않고, "thisisaword" 전체가 하나의 토큰이 되거나, "thisisa", "word" 등으로 잘못 분리될 수 있습니다.

In [ ]:
from tokenizers.processors import TemplateProcessing # TemplateProcessing은 토큰화된 시퀀스(sequence of tokens)에 특수 토큰(cls, sep, pad, mask, unk 등) 및 토큰 타입 ID(각 토큰이 몇번째 문장에 속하는 지)를 추가하는 후처리 역할을 합니다.
                                                     # BERT와 같은 트랜스포머 모델은 입력 텍스트를 특정한 형식으로 가공해야 제대로 작동하는데, TemplateProcessing은 이러한 형식화를 자동화
post_processor = TemplateProcessing(
    single="[CLS] $A [SEP]", #단일 문장(single sentence) 입력에 대한 템플릿
    pair="[CLS] $A [SEP] $B:1 [SEP]:1", #두 개의 문장(sentence pair) 입력에 대한 템플릿을 지정합니다.
    special_tokens=[(t, i) for i, t in enumerate(user_defined_symbols)] #위에 사용자 지정 특수 토큰 cell에서 정한 user_defined_symbols을 특수 토큰으로 지정
)                                                                       #[(t, i) for i, t in enumerate(...)]는 각 특수 토큰 t와 해당 토큰의 인덱스 i를 튜플 형태로 묶어 리스트를 만듭니다. (예: [('[PAD]', 0), ('[UNK]', 1), ('[CLS]', 2), ...])

bert_tokenizer.post_processor = post_processor

### 훈련설정 및 수행

In [ ]:
from tokenizers.trainers import WordPieceTrainer

vocab_size = 24000
trainer = WordPieceTrainer(
    vocab_size=vocab_size,#하이퍼 파라미터. 학습은 vocap size가 여기서 지정한 만큼 될 때까지 이뤄진다.
    special_tokens=whole_user_defined_symbols, #사용자 지정 특수 토큰 cell에서 정한 whole_user_defined_symbols (user_defined_symbols + unused_list)
)

In [ ]:
from glob import glob #glob 함수는 특정 패턴과 일치하는 파일 경로들을 찾아 리스트 형태로 반환하는 함수

bert_tokenizer.train(glob(f"/content/*.txt"), trainer) #glob을 사용하여 훈련 데이터 파일들을 가져와 bert_tokenizer.train()을 호출하여 훈련을 시작

### 훈련 결과 확인: 인코딩 및 디코딩

In [ ]:
##기본 디코더 사용: Subword들을 공백 없이 붙여서 출력, [CLS], [SEP] 등의 특수 토큰을 포함하여 출력

output = bert_tokenizer.encode("인코딩 및 디코딩이 제대로 이루어지는지 확인 중입니다.")
print(output.ids)

bert_tokenizer.decode(output.ids)

In [ ]:
##decoders.WordPiece(); bert_tokenizer.decode() 사용: Subword들 사이의 ## (접두사)를 제거하고, 적절히 공백을 넣어 출력, [CLS], [SEP] 등의 특수 토큰을 제거하고 출력

from tokenizers import decoders

bert_tokenizer.decoder = decoders.WordPiece()
bert_tokenizer.decode(output.ids)

In [ ]:
##Tokenizer 라이브러리로 만든 토크나이저를 트랜스포머 라이브러리에서 사용. 이렇게 하면 Tokenizer로 디테일하게 맞춤화한 걸 트랜스포머 라이브러리에서 불러와 사용 가능.

from transformers import BertTokenizerFast

fast_tokenizer = BertTokenizerFast(tokenizer_object=bert_tokenizer)
encoded = fast_tokenizer.encode("인코딩 및 디코딩이 제대로 이루어지는지 확인 중입니다.")
decoded = fast_tokenizer.decode(encoded)
print(encoded)
print(decoded)

### 훈련을 마친 토크나이저를 저장

In [ ]:
#저장

output_dir = "/content/MyTokenizer"
fast_tokenizer.save_pretrained(output_dir)

In [ ]:
#저장한 걸 불러와서 사용해본다

new_tokenizer = BertTokenizerFast.from_pretrained(output_dir)

encoded = new_tokenizer(["인코딩 잘 되는지 확인", "안되면 다시 학습하자"])

for k, v in encoded.items():
  print(k, v)

print(new_tokenizer.decode(encoded["input_ids"][0]))
print(new_tokenizer.decode(encoded["input_ids"][1]))